In [1]:
%cd ..

In [2]:
import numpy as np
import pandas as pd
import rx
from rx import operators
from rx import scheduler
from rx import subject

import logging
import datetime

from configs.config import GlobalConfig
GlobalConfig.initialize_global_configuration("configs/defaults.json")

from utils.logging import configure_logger_to_output
logging_output = configure_logger_to_output(level=logging.INFO)

# Description
Collect some image and Raman, FBRM information for 35 degc saturation
Solubility = 0.143003 g/g
water = 50 g
need taurine = 7.150 g

## Shared initialization

In [3]:
from utils.mqtt_wrapper import MQTTClientWrapper
from sources.harvesters_source import HarvestersSource
from sinks.save_image_sink import SaveImageSink
from sinks.save_data_sink import SaveDataSink
from sinks.visualization_sink import JupyterImageSink, PlotlyVisualizationSink
from datamodel.image import Image
from operators import data_framer
from plotly import graph_objects as go
from controls.camera_controller import CameraControl, CameraControlCommand
from controls.fp50 import FP50Command, FP50Control
from controls.mqtt_pump import MQTTPump
from sources.raman_source import RamanSource
from sources.mqtt_ds18 import MQTTDS18Source
from utils.auto_disposable import AutoDisposable

import ipywidgets as widgets


In [4]:
# MQTT connection
client = MQTTClientWrapper("experiment_taurine")
client.connect("192.168.43.1")
client.loop_start()

# DS18

In [5]:
ds18 = MQTTDS18Source(client)
ds18_save = SaveDataSink("ds18", "value", auto_timestamp = True)
ds18_save_subs = AutoDisposable(ds18.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"value": x["value"]}])),
).subscribe(ds18_save))
ds18_plot = PlotlyVisualizationSink(name="Temperature")
ds18_plot_subs = AutoDisposable(ds18.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"value": x["value"]}])),
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.value, name="Temperature")]),
).subscribe(ds18_plot))

## Water bath and temperature control

In [6]:
fp50 = FP50Control(client)
fp50_df = fp50.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"power": x.power, "setpoint": x.setpoint, "temperature": x.temperature}]))
)
fp50_save = SaveDataSink("fp50", "data", auto_timestamp = True)

fp50_save_subs = AutoDisposable(fp50_df.subscribe(fp50_save))
fp50_power_plot = PlotlyVisualizationSink(name="Power")
fp50_power_plot_subs = AutoDisposable(fp50_df.pipe(
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.power, name="Power")]),
).subscribe(fp50_power_plot))

fp50_temperature_plot = PlotlyVisualizationSink(name="Temperature")
fp50_temperature_plot_subs = AutoDisposable(fp50_df.pipe(
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.temperature, name="Temperature")]),
).subscribe(fp50_temperature_plot))

# Raman

In [7]:
# backend configuration
raman_source = RamanSource(client, "raman").pipe(operators.share())

raman_dataframe_source = raman_source.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            index=[datetime.datetime.now()], 
            data=[dict(zip(x["wave_number"], x["data"]))],
        )
    )
)

# save file logic
raman_save_sink = SaveDataSink("raman", "data")
raman_save_subscription = raman_dataframe_source.subscribe(raman_save_sink)

In [8]:
# front end configuration
raman_visuallization = PlotlyVisualizationSink(name="Raman")
raman_visuallization_subscription = raman_source.pipe(
    operators.map(lambda x: [go.Scatter(x=x["wave_number"], y=x["data"], )])
).subscribe(raman_visuallization)

raman_visuallization.figure.update_layout(xaxis={"autorange": "reversed", "title":"$Wave number (cm^{-1})$"}, yaxis={"title": "Count"})

;

''

# Concentrations

In [9]:
from properties.taurine import TaurineProperty
taurine_property = TaurineProperty(r"C:\Users\wuyua\OneDrive - The University of Western Ontario\Research\Taurine\Raman Calibration\data\raman_pca_linear_model.pkl")
def func(x):
    d = list(x[0]["data"])
    d.append(x[1]["value"])
    input_data = np.array(d).reshape((1, -1))
    y = taurine_property.raman_concentration(input_data)
    solids = y[0][0]
    solute = y[0][1]
    solubility = taurine_property.solubility(x[1]["value"])
    return {"solids": solids, "solute":solute, "solubility":solubility}
        
conc_src = raman_source.pipe(
    operators.with_latest_from(ds18),
    operators.map(func),
    operators.share()
)

# front end configuration

conc_plot = PlotlyVisualizationSink(name="Concentration")
conc_save = SaveDataSink("conc", "value", auto_timestamp = True)
conc_save_subs = AutoDisposable(conc_src.pipe(
    operators.map(lambda x: pd.DataFrame(data=[x])),
).subscribe(conc_save))
conc_plot_subs = AutoDisposable(conc_src.pipe(
    operators.map(lambda x: pd.DataFrame(data=[x])),
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x.solute, name="solute"), go.Scatter(x=x.timestamp, y=x.solubility, name="solubility"), go.Scatter(x=x.timestamp, y=x.solids, name="solids")]),
).subscribe(conc_plot))

# FBRM

In [10]:
from sources.fbrm_source import FBRMSource

# backend configuration
fbrm_source = FBRMSource(client, "fbrm").pipe(operators.share())

fbrm_dataframe_source = fbrm_source.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            index=[datetime.datetime.now()], 
            data=[dict(zip(x["sizes"], x["counts"]))],
        )
    )
)

# save file logic
fbrm_save_sink = SaveDataSink("fbrm", "data")
fbrm_save_subscription = fbrm_dataframe_source.subscribe(fbrm_save_sink)

In [11]:
# front end configuration
fbrm_visuallization = PlotlyVisualizationSink(name="FBRM")
fbrm_visuallization_subscription = fbrm_source.pipe(
    operators.map(lambda x: [go.Scatter(x=x["sizes"], y=x["counts"], )])
).subscribe(fbrm_visuallization)

fbrm_visuallization.figure.update_layout(xaxis={"type":"log", "title":"$Size (\mu m)$"}, yaxis={"title": "Count"})

fbrm_count_visuallization = PlotlyVisualizationSink(name="FBRM statistics")
fbrm_count_visuallization_subscription = fbrm_source.pipe(
    operators.map(lambda x: pd.DataFrame(index=[datetime.datetime.now()], data=[{
        "total": np.sum(x["counts"])
    }])),
    data_framer.data_framer(),
    operators.map(lambda x: [go.Scatter(x=x.index, y=x.total, )])
).subscribe(fbrm_count_visuallization)
;
fbrm_trend_visuallization = PlotlyVisualizationSink(name="FBRM Trend")
fbrm_visuallization_subscription = fbrm_source.pipe(
    operators.map(lambda x: pd.DataFrame(data=[{"total_count": sum(x["counts"])}])),
    data_framer.data_framer(window_length = 300, auto_timestamp = True),
    operators.map(lambda x: [go.Scatter(x=x.timestamp, y=x["total_count"], name="Total count")])
).subscribe(fbrm_trend_visuallization)

## Organize widgets

In [42]:
from ipywidgets import Layout, Box, VBox
# logging panel
# logging_output.layout = Layout(border="solid", width="100%", height="200px", overflow="scroll")

# temperature panel
temperature_panel = VBox(children=[fp50_power_plot.figure, fp50_temperature_plot.figure, ds18_plot.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)

# FBRM panel
fbrm_panel = VBox(children=[fbrm_visuallization.figure, fbrm_trend_visuallization.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)


# Raman Panel
raman_panel = Box(children=[raman_visuallization.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)

# Concentration panel
conc_panel = Box(children=[conc_plot.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)


# control panel
event_logger_text = widgets.Text(
    description='Event annotation:',
    layout=Layout(width="100%"),
)
event_logger_text.on_submit(lambda x: logging.getLogger("annotation").info(x.value))


control_panel = Box(children=[event_logger_text], layout=Layout(border="solid", width="100%"))

# wrap up
panel = widgets.VBox([temperature_panel, fbrm_panel, raman_panel, conc_panel, control_panel])

display(panel)

    'data': [{'name': 'Power',
              'type': 'scatter',
 …

In [41]:
delta_t = 5
start =24.9
cooling_rate = -0.04/60 # d/s
end = 25.2

subs = rx.interval(delta_t).pipe(
    operators.map(lambda x: start - x*delta_t * cooling_rate),
    operators.take_while(lambda x: x < end),
    operators.map(lambda x: FP50Command(round(x, 2))),
).subscribe(lambda x: fp50.on_command(x))

In [40]:
subs.dispose()

In [63]:
fp50.on_command(FP50Command(32.3))